#### Download dataset

In [ ]:
!pip install --upgrade gdown
!gdown --fuzzy 1B6inr_JHXRTidFsSMqOGhJWJr_09hXdF

In [ ]:
!tar xf where_am_i.tar

In [ ]:
import tensorflow as tf
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from sklearn.model_selection import train_test_split

#### Hyperparameters

In [ ]:
IMG_SIZE = 224
BS = 64

In [ ]:
# 建立類別名稱轉class idx字典
with open("mapping.txt") as f:
    lines = f.readlines()
lines = [l.strip().replace(" ", "") for l in lines]
CLASS_MAP = {pair.split(",")[0]: int(pair.split(",")[1]) for pair in lines}
NUM_CLASS = len(CLASS_MAP)

CLASS_MAP, NUM_CLASS

#### Build Dataset

In [ ]:
# 撈取有解答之圖片路徑, 分割train, val路徑
paths = glob("train/*/*.jpg")
train_paths, val_paths = train_test_split(paths, 
                                          test_size=0.2, 
                                          random_state=5566)

len(train_paths), len(val_paths)

In [ ]:
# 圖片視覺化
path = np.random.choice(train_paths)
img = np.array(Image.open(path).convert("RGB"))
plt.imshow(img)
img.shape, path

In [ ]:
# 擷取資料夾名稱轉成類別id
# os.sep: "/"
def paths2labels(paths):
    return [CLASS_MAP[p.split(os.sep)[-2]] for p in paths]

In [ ]:
train_paths[:10], paths2labels(train_paths)[:10]

In [ ]:
# 影像讀取 & resize
def load_and_resize_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image

# 使用路徑建構 tf.data.Dataset
def build_ds(paths):
    labels = paths2labels(paths) # paths -> labels
    image_ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    image_ds = image_ds.map(lambda path, label: (load_and_resize_image(path), label)) # path -> img, labels
    return image_ds

In [ ]:
# 建構train, val dataset
train_ds = build_ds(train_paths).shuffle(buffer_size=len(train_paths)).batch(BS)
val_ds = build_ds(val_paths).batch(BS)

#### Build Model

In [ ]:
class WAIModel(tf.keras.models.Model):
    def __init__(self, encoder, preprocess, freeze):
        super().__init__()
        # 前處理函數
        self.preprocess = preprocess
        # 資料擴增
        self.data_aug = tf.keras.Sequential([
            tf.keras.layers.RandomFlip("horizontal"),
            tf.keras.layers.RandomRotation(0.1),
        ])
        # feature extractor
        self.encoder = encoder
        # classifier
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(NUM_CLASS),
            tf.keras.layers.Softmax(),
        ])
        # freeze: 是否鎖住encoder參數
        if freeze:
            for l in self.encoder.layers:
                l.trainable = False
    def call(self, inputs, training=None):
        x = self.preprocess(inputs)
        # 非訓練時不使用資料擴增
        if training:
            x = self.data_aug(x)
            
        x = self.encoder(x, training)
        x = self.classifier(x, training)
        return x

#### Use pre-trained model

https://keras.io/api/applications/

https://www.tensorflow.org/api_docs/python/tf/keras/applications

In [ ]:
# 使用模型對應之前處理函式
preprocess = tf.keras.applications.efficientnet.preprocess_input
encoder = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, 
                                                            weights='imagenet')

model = WAIModel(encoder, 
                 preprocess,
                 freeze=True)

In [ ]:
model.encoder.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

#### Training

In [ ]:
model.fit(train_ds, 
          epochs=20, 
          validation_data=val_ds,
          callbacks=[
              tf.keras.callbacks.ModelCheckpoint("best.h5",
                                                 save_best_only=True,
                                                 save_weights_only=True)
          ])

#### Test submission

繳交至：https://www.kaggle.com/competitions/aia-xt121-cv-kaggle

In [ ]:
import pandas as pd

In [ ]:
# 讀取最佳val_loss model參數
model.load_weights("best.h5")

In [ ]:
df = pd.read_csv("sample_submission.csv")

In [ ]:
df.head(), len(df)

In [ ]:
# 建構測試dataset
test_paths = [os.path.join(f"test/{name}.jpg") for name in df["id"].tolist()]
test_ds = tf.data.Dataset.from_tensor_slices((test_paths))
test_ds = test_ds.map(lambda path: load_and_resize_image(path)).batch(BS)

In [ ]:
prediction = model.predict(test_ds)
prediction = np.argmax(prediction, axis=-1)

In [ ]:
df["class"] = prediction
df.to_csv("submission.csv", index=None)

Public  Private

0.92264 0.90529

#### Freeze some layers

In [ ]:
tf.keras.backend.clear_session()
encoder = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, 
                                                            weights='imagenet')

In [ ]:
for i, layer in enumerate(encoder.layers):
    print(i, layer.trainable, layer.name)

In [ ]:
for layer in encoder.layers[:100]:
    layer.trainable = False

In [ ]:
encoder.summary()